In [1]:
import pandas as pd
from pandas.io.json import json_normalize
from pandas import DataFrame
import os
os.getcwd()
from geopy import geocoders
from geopy.geocoders import GoogleV3
from config import gkey

In [2]:
import geopandas as gpd
from shapely import wkt

fname = "../../data/addresses/us_il_city_of_chicago-addresses-state.geojson"

gdf = gpd.read_file(fname)
addressdf = pd.DataFrame(gdf)
addressdf['geometry'] = addressdf.geometry.apply(lambda x: wkt.dumps(x))

In [3]:
#print(type(addressdf))
addressdf

,id,unit,number,street,city,district,region,postcode,hash,geometry
0,,,7726,W JARLATH ST,,,,,2cac3b0c78b34539,POINT (-87.8200365999999946 42.0125603999999981)
1,,,7724,W JARLATH ST,,,,,08fc96e7f8cefc4d,POINT (-87.8198816000000022 42.0125639000000035)
2,,,7720,W JARLATH ST,,,,,e3ad1acd7a000511,POINT (-87.8197057999999942 42.0125677999999994)
3,,,7714,W JARLATH ST,,,,,d63788ef9552e1da,POINT (-87.8195391999999941 42.0125715000000000)
4,,,7248,N OVERHILL AVE,,,,,5f66e762c70f7374,POINT (-87.8191652999999945 42.0128339999999980)
...,...,...,...,...,...,...,...,...,...,...
605655,,,1870,N DAMEN AVE,,,,,9c880cac52796c98,POINT (-87.6778917000000035 41.9158554999999993)
605656,,,5014,S KING DR,,,,,72b4137c242c38f7,POINT (-87.6170838000000032 41.8033525000000026)
605657,,,2236,W BELMONT AVE,,,,,672db80a65ce1a2b,POINT (-87.6847611000000029 41.9397601000000009)
605658,,,2318,S CANAL ST,,,,,81dc4df792504c55,POINT (-87.6388280999999978 41.8502095999999995)


In [5]:
# Combine address columns to one column
addressdf['addresses'] = addressdf['number'].str.cat(addressdf['street'] + ", Chicago, Illinois",sep=" ")
addressdf['geometry'] = addressdf['geometry'].str.replace('POINT ', '').str.replace('(', '').str.replace(')', '')
addressdf

,id,unit,number,street,city,district,region,postcode,hash,geometry,addresses
0,,,7726,W JARLATH ST,,,,,2cac3b0c78b34539,-87.8200365999999946 42.0125603999999981,"7726 W JARLATH ST, Chicago, Illinois"
1,,,7724,W JARLATH ST,,,,,08fc96e7f8cefc4d,-87.8198816000000022 42.0125639000000035,"7724 W JARLATH ST, Chicago, Illinois"
2,,,7720,W JARLATH ST,,,,,e3ad1acd7a000511,-87.8197057999999942 42.0125677999999994,"7720 W JARLATH ST, Chicago, Illinois"
3,,,7714,W JARLATH ST,,,,,d63788ef9552e1da,-87.8195391999999941 42.0125715000000000,"7714 W JARLATH ST, Chicago, Illinois"
4,,,7248,N OVERHILL AVE,,,,,5f66e762c70f7374,-87.8191652999999945 42.0128339999999980,"7248 N OVERHILL AVE, Chicago, Illinois"
...,...,...,...,...,...,...,...,...,...,...,...
605655,,,1870,N DAMEN AVE,,,,,9c880cac52796c98,-87.6778917000000035 41.9158554999999993,"1870 N DAMEN AVE, Chicago, Illinois"
605656,,,5014,S KING DR,,,,,72b4137c242c38f7,-87.6170838000000032 41.8033525000000026,"5014 S KING DR, Chicago, Illinois"
605657,,,2236,W BELMONT AVE,,,,,672db80a65ce1a2b,-87.6847611000000029 41.9397601000000009,"2236 W BELMONT AVE, Chicago, Illinois"
605658,,,2318,S CANAL ST,,,,,81dc4df792504c55,-87.6388280999999978 41.8502095999999995,"2318 S CANAL ST, Chicago, Illinois"


In [8]:
addressdf.drop(['unit', 'number', 'street', 'region', 'city', 'district', 'hash'],axis=1,inplace=True)
addressdf['addresses'] = addressdf['addresses'].astype('|S')
addressdf.dtypes


id           object
postcode     object
geometry     object
addresses    object
dtype: object

In [10]:
adds_sml = addressdf.sample(n = 100) 
adds_sml

,id,postcode,geometry,addresses
242801,,,-87.7702404999999999 41.8694769000000022,"b'921 S MONITOR AVE, Chicago, Illinois'"
25439,,,-87.8326832000000053 41.9530353000000034,"b'4027 N PITTSBURGH AVE, Chicago, Illinois'"
189835,,,-87.6613107000000014 41.9227379000000013,"b'1309 W BELDEN AVE, Chicago, Illinois'"
234958,,,-87.7055694000000017 41.8504294999999971,"b'2244 S KEDZIE AVE, Chicago, Illinois'"
165386,,,-87.6768799999999970 41.9571905000000029,"b'1908 W WARNER AVE, Chicago, Illinois'"
...,...,...,...,...
58780,,,-87.7285023999999964 41.9722198999999989,"b'5006 N PULASKI RD, Chicago, Illinois'"
175881,,,-87.6473715999999996 41.9324109000000007,"b'719 W DIVERSEY PKWY, Chicago, Illinois'"
511225,,,-87.6986227999999954 41.6882248000000004,"b'11252 S ALBANY AVE, Chicago, Illinois'"
488198,,,-87.5754263000000037 41.7400063999999986,"b'8521 S JEFFERY BLVD, Chicago, Illinois'"


In [ ]:
addresses = ["2620 N Drake Ave, Chicago, Illinois 60647",
         "2622 N Drake Ave, Chicago, Illinois 60647",
         "2626 N Drake Ave, Chicago, Illinois 60647",
         "2630 N Drake Ave, Chicago, Illinois 60647",
         "2648 N Drake Ave, Chicago, Illinois 60647"]

In [ ]:
df = DataFrame (addresses, columns=['Address'])
df

In [ ]:
API_KEY = os.getenv("API1234")
g = GoogleV3(api_key=gkey)


loc_coordinates = []
loc_address = []
for address in df.Address:
    try:
        inputAddress = address
        location = g.geocode(inputAddress, timeout=15)
        loc_coordinates.append((location.latitude, location.longitude))
        loc_address.append(inputAddress)
    except Exception as e:
        print('Error, skipping address...', e)


df_geocodes = pd.DataFrame({'coordinate':loc_coordinates,'address':loc_address})
df_geocodes

In [ ]:
import urllib.request, urllib.parse 
import requests
from requests.utils import requote_uri
import json
# Import google_streetview for the api module
import google_streetview.api


# Define parameters for street view api
params = [{
    'size': '600x300', # max 640x640 pixels
    'location': '41.9295582,-87.7159139',
    'heading': '90',
    'pitch': '-0.76',
    'key': gkey
}]

# Create a results object
results = google_streetview.api.results(params)

# Download images to directory 'downloads'
results.download_links('static/data/raw')

In [ ]:
# GRAB A SERIES OF STREET VIEW IMAGES FROM ADDRESSES PROVIDED
'''
myloc = "static/data/raw"
key = key = "&key=" + "gkey"
def GetStreet(Add,SaveLoc):
    base = "https://maps.googleapis.com/maps/api/streetview?size=1200x800&location="
    #MyUrl = base + urllib.parse.quote_plus(Add) + key  #added url encoding
    MyUrl = base + requote_uri(Add) + key  #added url encoding
    fi = Add + ".jpg"
    requests.get(MyUrl, os.path.join(SaveLoc,fi), stream=True, headers={'User-agent': 'Mozilla/5.0'})
  

Tests = ["2620 N Drake Ave, Chicago, Illinois 60647",
         "2622 N Drake Ave, Chicago, Illinois 60647",
         "2626 N Drake Ave, Chicago, Illinois 60647",
         "2630 N Drake Ave, Chicago, Illinois 60647",
         "2648 N Drake Ave, Chicago, Illinois 60647"]

for i in Tests:
 GetStreet(Add=i,SaveLoc=myloc)
''' 
  
    

In [ ]:

'''
# load and show an image with Pillow

# load the image
image = Image.open('static/data/test/brickornot.jpg')
# summarize some details about the image
print(image.format)
print(image.mode)
print(image.size)
# show the image
image.show()'''

In [ ]:
'''# load all images in a directory
loaded_images = list()
for filename in listdir('images'):
	# load image
	img_data = image.imread('images/' + filename)
	# store loaded image
	loaded_images.append(img_data)
	print('> loaded %s %s' % (filename, img_data.shape))'''


In [ ]:
%matplotlib inline
import PIL
from PIL import Image
from os import listdir
from matplotlib import image
from matplotlib import pyplot

# example of saving a grayscale version of a loaded image
from PIL import Image
# load the image
image = Image.open('static/data/test/brickornot.jpg')
# convert the image to grayscale
gs_image = image.convert(mode='L')
# report the size of the image
print(image.size)
# create a thumbnail and preserve aspect ratio
image.thumbnail((100,100))
# report the size of the thumbnail
print(image.size)
# save in jpeg format
gs_image.save('static/data/test_convert/brickornot.jpg')

In [ ]:
# load the image again and show it
image2 = Image.open('static/data/test_convert/brickornot.jpg')
# show the image
image2.show()

In [ ]:
'''# save as PNG format
image.save('opera_house.png', format='PNG')
# load the image again and inspect the format
image2 = Image.open('opera_house.png')
print(image2.format)'''